## 2. This Jupyter Notebook shows the second iteration, still using overall rank as target. All columns with the word "rank" in them were removed from consideration as features.

In [1]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import r2_score


In [2]:
# Loading data
file_path = Path("Resources/2022-College-Guide-Main-Rankings-clear.csv")
df_rankings = pd.read_csv(file_path)
df_rankings.head()


,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank,Unnamed: 40
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,0.290903,10,NO,3,0.068361,401,3,2,187,NaN
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,0.160573,71,YES,6,0.246743,209,2,9,40,NaN
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,0.065477,332,NO,2,0.000000,436,5,1,324,NaN
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,0.064500,339,NO,6,0.000000,436,7,6,224,NaN
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,0.089523,224,YES,5,0.170164,305,12,15,20,NaN


In [3]:
df_rankings.columns

Index(['UnitID', 'Rank', 'Name', '8-year graduation rate',
       'Graduation rate rank',
       'Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.',
       'Graduation rate performance rank', 'Pell/non-Pell graduation gap',
       'Pell graduation gap rank', 'Number of Pell graduates',
       'Actual vs. predicted Pell enrollment', 'Pell performance rank',
       'Median earnings 10 years after entering college',
       'Predicted median earnings 10 years after entering college',
       'Earnings performance rank',
       'Net price of attendance for families below $75,000 income',
       'Net price rank', '% of loan principal remaining 5 years later',
       'Repayment rank', 'Predicted principal remaining',
       'Repayment rate performance rank', 'Research expenditures, in millions',
       'Research expenditures rank', 'Bachelor's to PhD rank',
       'Science & engineering PhDs awarded', 'Science & engineering PhDs rank',
       'Faculty receiving sign

In [4]:
df_rankings.dtypes

UnitID                                                                            int64
Rank                                                                              int64
Name                                                                             object
8-year graduation rate                                                          float64
Graduation rate rank                                                              int64
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    float64
Graduation rate performance rank                                                  int64
Pell/non-Pell graduation gap                                                    float64
Pell graduation gap rank                                                          int64
Number of Pell graduates                                                        float64
Actual vs. predicted Pell enrollment                                            float64
Pell performance rank           

In [5]:
df_rankings.count()

UnitID                                                                          442
Rank                                                                            442
Name                                                                            442
8-year graduation rate                                                          442
Graduation rate rank                                                            442
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    442
Graduation rate performance rank                                                442
Pell/non-Pell graduation gap                                                    442
Pell graduation gap rank                                                        442
Number of Pell graduates                                                        442
Actual vs. predicted Pell enrollment                                            442
Pell performance rank                                                       

In [6]:
# Drop last column
df_rankings.drop('Unnamed: 40', axis=1, inplace=True)

In [7]:
# Encoding Carnegie column (0=NO, 1=YES)

le = LabelEncoder()
df2_rankings = df_rankings.copy()
df2_rankings['Earns Carnegie Community Engagement Classification?'] = le.fit_transform(df2_rankings['Earns Carnegie Community Engagement Classification?'])


In [8]:
df2_rankings.head()

,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,308,0.290903,10,0,3,0.068361,401,3,2,187
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,250,0.160573,71,1,6,0.246743,209,2,9,40
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,12,0.065477,332,0,2,0.000000,436,5,1,324
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,34,0.064500,339,0,6,0.000000,436,7,6,224
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,45,0.089523,224,1,5,0.170164,305,12,15,20


In [9]:
# Define the features set.
X = df2_rankings.copy()

# Dropping target columns and unique identifier columns, and ALL RANK COLUMNS
X = X.drop(columns=['UnitID', 'Rank', 'Name', 'Graduation rate rank', 'Graduation rate performance rank', 'Pell graduation gap rank', 
     'Pell performance rank',
     'Earnings performance rank',
     'Net price rank', 'Repayment rank', 'Repayment rate performance rank',
     'Research expenditures rank', "Bachelor's to PhD rank",
     'Science & engineering PhDs rank', 'Faculty accolades rank', 'AmeriCorps/Peace Corps rank', 'ROTC rank',
     '% of federal work-study funds spent on service rank',
     'Service-oriented majors rank', 'Social mobility rank', 'Research rank',
     'Service rank'])
X.head()


,8-year graduation rate,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Pell/non-Pell graduation gap,Number of Pell graduates,Actual vs. predicted Pell enrollment,Median earnings 10 years after entering college,Predicted median earnings 10 years after entering college,"Net price of attendance for families below $75,000 income",% of loan principal remaining 5 years later,Predicted principal remaining,"Research expenditures, in millions",Science & engineering PhDs awarded,Faculty receiving significant awards,Faculty in National Academies,% of federal work-study funds spent on service,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors
0,0.963302,0.949654,-0.021703,260.6667,0.038096,97798,86131.63,1445.8400,0.322153,0.383026,1188.5540,638.3333,0.044643,0.292627,0.290903,0,3,0.068361
1,0.964294,0.938620,-0.041917,376.6667,-0.000590,103246,82677.28,7368.1700,0.351408,0.401545,1509.1930,328.3333,0.019748,0.076843,0.160573,1,6,0.246743
2,0.950971,0.990213,-0.032985,182.6667,0.076769,111222,95700.68,3290.8450,0.305619,0.344599,987.2567,599.0000,0.028059,0.267684,0.065477,0,2,0.000000
3,0.976861,0.947812,-0.052740,153.3333,0.062997,95689,82261.91,3672.0020,0.360991,0.388965,385.2780,278.0000,0.026616,0.118821,0.064500,0,6,0.000000
4,0.954474,0.967952,-0.014278,231.6667,0.028513,93115,85402.80,671.8563,0.361023,0.360074,1196.9220,320.6667,0.014617,0.048649,0.089523,1,5,0.170164


In [10]:
# Checking datatypes to make sure there are no strings
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 18 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   8-year graduation rate                                                        442 non-null    float64
 1   Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.  442 non-null    float64
 2   Pell/non-Pell graduation gap                                                  442 non-null    float64
 3   Number of Pell graduates                                                      442 non-null    float64
 4   Actual vs. predicted Pell enrollment                                          442 non-null    float64
 5   Median earnings 10 years after entering college                               442 non-null    int64  
 6   Predicted median earnings 10 years

In [11]:
# Define the target set.
y = df2_rankings["Rank"].ravel()
y[:5]


array([1, 2, 3, 4, 5], dtype=int64)

In [12]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [13]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (331, 18)
Training Labels Shape: (331,)
Testing Features Shape: (111, 18)
Testing Labels Shape: (111,)


## Random Forest Regressor

In [14]:
# Instantiate model with 1000 decision trees
rf_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf_model=rf_model.fit(X_train_scaled, y_train)

In [15]:
# Making predictions using the testing data.
rf_predictions = rf_model.predict(X_test_scaled)


In [16]:
# Evaluate the performance with r2 score
r2_rf= r2_score(y_test,rf_predictions)
print('The r2 score for this Random Forest Regressor model is:', r2_rf)

The r2 score for this Random Forest Regressor model is: 0.7704453127500439


In [17]:
# Getting accuracy data with Regressor/continuous variable model

# Calculate the absolute errors
errors = abs(rf_predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Mean Absolute Error: 45.16
Accuracy: 58.58 %.


In [18]:
y_test

array([340,  46, 201, 374, 111, 355, 295, 169, 303,  84,  70,  68, 140,
       160, 222, 434, 215, 133,   7,  31, 276, 336, 263, 314,  99, 370,
       227, 273, 206, 126, 327, 250, 422, 284, 142, 192, 136, 436, 313,
       428, 168, 171, 337, 230, 204, 367, 157,  25, 292,  64, 262, 196,
         8, 254, 299,  75, 368, 361, 187,  22, 259,  93, 256,  16, 363,
       184, 249,  33,  20, 366,   3,  95, 210,  52, 182, 281, 128,  92,
       382, 349,  49, 411, 158, 224, 383, 421, 387, 351,  51, 195, 296,
        17, 274, 202,  88, 241,  21,  91, 424,  76, 137, 258,  42, 243,
       392, 309,  72, 401, 350, 233, 221], dtype=int64)

In [19]:
errors

array([ 64.072,  65.194,   8.919,  42.609,  77.316,  29.651,   4.675,
        73.525,  40.313,  44.118,  38.641,  90.882,  96.666,  11.583,
         9.763,  82.273, 106.427,  22.564,   3.795,  11.713,  14.53 ,
        36.565,   2.014,  49.073,  21.782,  93.276,  31.015,  10.724,
        25.138,  42.317,   9.5  ,  10.207,  36.665,  59.597,  98.295,
        85.438,  36.588, 122.143,  32.641,  55.27 ,  44.023,  16.245,
        22.876,   8.548,  27.187, 113.334,  46.084,  20.525,  24.873,
       163.737, 122.063,  45.862,   9.469,  26.551,  15.011,  52.073,
        33.304, 251.85 ,  96.728, 124.234,  24.331,  33.054,   7.222,
        21.743,  34.381,  12.369,  82.087,  21.673,   5.423,  62.18 ,
         9.653,   7.595,  16.889,  27.557,  58.987,   2.977,  34.611,
        70.196,  60.053, 128.109,  45.64 ,  99.81 ,  47.7  ,  15.879,
        53.384,  33.585,  14.222,  14.079, 111.317,   6.637,  22.992,
        20.391,   5.45 ,  65.62 ,  10.438,  95.379,   9.078,  24.727,
        68.127,  27.

In [20]:
from sklearn import metrics
# https://machinelearningmastery.com/regression-metrics-for-machine-learning

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, rf_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, rf_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, rf_predictions)))

Mean Absolute Error: 45.15621621621621
Mean Squared Error: 3570.3535363243245
Root Mean Squared Error: 59.752435400779476


In [22]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


array([0.16907209, 0.01161832, 0.08058771, 0.03025824, 0.03338201,
       0.03810903, 0.01544687, 0.13711117, 0.07112368, 0.01093481,
       0.02157328, 0.01417343, 0.28281265, 0.01154132, 0.01652839,
       0.01453058, 0.02530778, 0.01588863])

In [24]:
# Sort the features by their importance.
sorted_list=sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
sorted_df=pd.DataFrame(sorted_list, columns=['Importance', 'Feature'])
sorted_df

,Importance,Feature
0,0.282813,Faculty receiving significant awards
1,0.169072,8-year graduation rate
2,0.137111,Net price of attendance for families below $75...
3,0.080588,Pell/non-Pell graduation gap
4,0.071124,% of loan principal remaining 5 years later
5,0.038109,Median earnings 10 years after entering college
6,0.033382,Actual vs. predicted Pell enrollment
7,0.030258,Number of Pell graduates
8,0.025308,Voting engagement points
9,0.021573,"Research expenditures, in millions"


In [25]:
sorted_df.to_csv('Feature_Importances_with_all_RANK_features_removed.csv')  

## Support Vector Regression

In [31]:
# Create a SVM model

# Instantiate a SVM model
from sklearn.svm import SVR

## the kernel is the most important part - try different kernels

# Fit the data

regressor = SVR(kernel = 'poly')
svm_model = regressor.fit(X_train, y_train)
svm_model


SVR(kernel='poly')

In [32]:
# Predicting the Test set results
svm_predictions = svm_model.predict(X_test)

In [33]:
from sklearn.metrics import r2_score
r2_svm=r2_score(y_test, svm_predictions)
print('The r2 score for this SVM model is:', r2_svm)


The r2 score for this SVM model is: 0.3220156109059724


## Decision Tree Regression

In [35]:
# Training the Decision Tree Regression model on the Training set
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state = 0)
dt_model = dt_model.fit(X_train, y_train)
dt_model

DecisionTreeRegressor(random_state=0)

In [36]:
# Predicting the Test set results
dt_predictions = dt_model.predict(X_test)

In [37]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_dt= r2_score(y_test,dt_predictions)
print('The r2 score for this Decision Tree model is:', r2_dt)

The r2 score for this Decision Tree model is: 0.5743310834322775


## Multiple Linear Regression

In [38]:
# Training the multiple regression model on the Training set
from sklearn.linear_model import LinearRegression
mlr_model = LinearRegression()
mlr_model = mlr_model.fit(X_train, y_train)
mlr_model

LinearRegression()

In [39]:
# Predicting the Test set results
mlr_predictions = mlr_model.predict(X_test)

In [40]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_mlr= r2_score(y_test,mlr_predictions)
print('The r2 score for this Multiple Linear Regression model is:', r2_mlr)

The r2 score for this Multiple Linear Regression model is: 0.9238440095251319
